## 安装必要的库
> 请提前准备好mindspore和mindnlp的安装

首先，安装所需的Python库：
- `-q` 表述静默安装，不会出现很多的`Requirement already satisfied`等等

In [1]:
!pip install -q huggingface_hub ipywidgets opencv-python

You should consider upgrading via the '/home/ma-user/anaconda3/envs/MindSpore/bin/python3.9 -m pip install --upgrade pip' command.


## 设置环境变量
设置Hugging Face的国内镜像站点以加快下载速度：

In [2]:
import os
os.environ["HF_ENDPOINT"] = "https://hf-mirror.com"       

## 下载加载视频
- 使用`huggingface_hub`下载视频文件
- 使用`ipywidgets`展示视频

In [3]:
from huggingface_hub import hf_hub_download
from ipywidgets import Video

file_path = hf_hub_download(
    repo_id="nielsr/video-demo", filename="eating_spaghetti.mp4", repo_type="dataset"
)
Video.from_file(file_path, width=500)

Video(value=b'\x00\x00\x00 ftypisom\x00\x00\x02\x00isomiso2avc1mp41\x00\x00\x00\x08free...', width='500')

## 定义采样函数
`sample_frame_indices`通过在给定的帧范围内随机选择一段视频片段，并返回这段片段的帧索引。
- `clip_len`: 需要采样的帧数。
- `frame_sample_rate`: 帧采样率，决定了采样的密度。
- `seg_len`: 视频的总帧数。

In [4]:
import numpy as np

np.random.seed(0)

def sample_frame_indices(clip_len, frame_sample_rate, seg_len):
    # 计算转换后的长度(在给定采样率下，实际需要的帧数长度)
    converted_len = int(clip_len * frame_sample_rate)
    # 选择结束索引
    end_idx = np.random.randint(converted_len, seg_len)
    # 计算开始索引
    start_idx = end_idx - converted_len
    #使用np.linspace在开始索引和结束索引之间生成clip_len个等间距的索引
    indices = np.linspace(start_idx, end_idx, num=clip_len)
    # 使用np.clip确保所有索引都在有效范围内，并将它们转换为整数类型
    indices = np.clip(indices, start_idx, end_idx - 1).astype(np.int64)
    return indices

## 读取视频帧
利用OpenCV

In [5]:
import cv2
def read_video(file_path, indices):
    # 打开视频文件
    cap = cv2.VideoCapture(file_path)
    # 初始化一个列表来存储帧
    frames = []
    # 遍历给定的帧索引
    for idx in indices:
        # 设置视频捕获对象到特定的帧位置
        cap.set(cv2.CAP_PROP_POS_FRAMES, idx)
        # 读取该帧
        ret, frame = cap.read()
        # 将读取的帧添加到帧列表中，并且转换通道，因为opencv是BGR
        if ret:
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)  # 转换为RGB
            frames.append(frame)
    # 释放视频捕获对象
    cap.release()
    # 将帧列表转换为NumPy数组并返回
    return np.array(frames)


## 采样和读取视频
采样8帧并读取：

In [6]:
cap = cv2.VideoCapture(file_path)
seg_len = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
indices = sample_frame_indices(clip_len=8, frame_sample_rate=1, seg_len=seg_len)
video = read_video(file_path, indices)
video.shape

(8, 360, 640, 3)

## 使用mindnlp库进行文本-视频匹配

本次我们使用的是`X-CLIP`，一个将语言-图像基础模型适配于通用视频识别的新框架。
- `X-CLIP`的整体结构与`CLIP`相似，采用两个编码器分别对文本和视频进行编码，然后通过比对这些特征来实现分类。
- `X-CLIP`引入了一个轻量级、可“即插即用”的跨帧注意力模块，用于捕捉时间信息。
- 此外，该模型使用视频提示（Prompt），可以生成具有区分能力的视觉提示，从而提升分类效果。因此，无需额外数据，`X-CLIP` 有效地利用了预训练的语言-图像模型，通过零样本或少样本学习实现视频识别。

论文信息：
> [**Expanding Language-Image Pretrained Models for General Video Recognition**](https://arxiv.org/abs/2208.02816)<br>
> accepted by ECCV 2022 as an oral presentation<br>
> Bolin Ni, [Houwen Peng](https://houwenpeng.com/), [Minghao Chen](https://silent-chen.github.io/), [Songyang Zhang](https://sy-zhang.github.io/), [Gaofeng Meng](https://people.ucas.ac.cn/~gfmeng), [Jianlong Fu](https://jianlong-fu.github.io/), [Shiming Xiang](https://people.ucas.ac.cn/~xiangshiming), [Haibin Ling](https://www3.cs.stonybrook.edu/~hling/)

[[arxiv]](https://arxiv.org/abs/2208.02816)
[[slides]](https://github.com/nbl97/X-CLIP_Model_Zoo/releases/download/v1.0/xclip-slides.pptx)


In [7]:
from mindnlp.transformers import XCLIPProcessor, XCLIPModel
model_name = "microsoft/xclip-base-patch32"
processor = XCLIPProcessor.from_pretrained(model_name)
model = XCLIPModel.from_pretrained(model_name)

/home/ma-user/anaconda3/envs/MindSpore/lib/python3.9/site-packages/numpy/core/getlimits.py:499: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/home/ma-user/anaconda3/envs/MindSpore/lib/python3.9/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  return self._float_to_str(self.smallest_subnormal)
/home/ma-user/anaconda3/envs/MindSpore/lib/python3.9/site-packages/numpy/core/getlimits.py:499: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/home/ma-user/anaconda3/envs/MindSpore/lib/python3.9/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  return self._float_to_str(self.smallest_subnormal)
Building prefix dict from the default dict

[MS_ALLOC_CONF]Runtime config:  enable_vmm:True  vmm_align_size:2MB


[WARNING] CORE(90475,ffff864fc0b0,python):2024-10-24-20:59:31.816.322 [mindspore/core/utils/ms_context.cc:531] GetJitLevel] Set jit level to O2 for rank table startup method.


## 设置提示词和输入

In [8]:
inputs = processor(text=["playing sports", "eating spaghetti", "go shopping"], videos=list(video), return_tensors="ms")

## 前向计算

In [9]:
outputs = model(**inputs)

In [10]:
logits_per_video = outputs.logits_per_video  # 这是视频-文本相似度得分
logits_per_video

Tensor(shape=[1, 3], dtype=Float32, value=
[[ 1.26835327e+01,  2.11186066e+01,  1.28016310e+01]])

In [11]:
# 我们可以使用 softmax 来获取标签概率
from mindspore import ops
ops.softmax(logits_per_video,1)

Tensor(shape=[1, 3], dtype=Float32, value=
[[ 2.17016917e-04,  9.99538779e-01,  2.44221010e-04]])